In [336]:
import numpy as np
import pandas as pd

# Setup Environmental Flow Requirements

Based on the environmental flow bands, the methodology adopted for the energy modelling is as below:

- The flow on the 8th, 18th and 28th day of each month is identified. 
- The equivalent percentile of this flow based on flow exceedance curves is calculated for that day from the 92 years of data.
- This percentile is used to decide the environmental flow release for the succeeding ~10 day periods of 1st to 10th, 11th to 20th and 21st to end of the month.
- Environmental flow in this band and month is released in the respective channels.

In this notebook the exceedance percentiles for each day of the historic flow series are calculated. These will later be used to lookup the required EWR values for each channel depending on the Environmental Assurance Category selected for the channel. 

In [337]:
#Gives the measure date (proceeding 8th, 18th or 28th) for the given date
def measureDate(obsDate):
    if obsDate.day <= 10:
        if obsDate.month==1:
            mn=13
            yr=obsDate.year-1
        else:
            mn=obsDate.month
            yr=obsDate.year
        return pd.Timestamp(yr, mn-1, 28)
    elif obsDate.day <=20:
        return pd.Timestamp(obsDate.year, obsDate.month, 8)
    else:
        return pd.Timestamp(obsDate.year, obsDate.month, 18)

#If it is a measure date (8th, 18th or 28th) then returns the ID of an FDC for that date. Otherwise False.
def isMeasureDay(obsDate):
    if obsDate.day==8 or obsDate.day==18 or obsDate.day==28:
        return str(obsDate.month) + '-' + str(obsDate.day) 
    else:
        return False
    
#Given an FDC ID and flow looks up exceedance
def lookupExceedance(measureDay,flow):
    return ewr_fdcs[ewr_fdcs[measureDay]<=flow].index.min()

In [345]:
daily=pd.read_csv("prepared_daily.csv")
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)

Build FDCs for each of the 'measurement days' - 8th, 18th, 28th of each month

In [346]:
cols=['9-28','10-8','10-18','10-28','11-8','11-18','11-28','12-8','12-18','12-28',
      '1-8','1-18','1-28','2-8','2-18','2-28','3-8','3-18','3-28','4-8','4-18',
      '4-28','5-8','5-18','5-28','6-8','6-18','6-28','7-8','7-18','7-28','8-8','8-18','8-28','9-8','9-18']
ewr_fdcs=pd.DataFrame(columns=cols,index=np.arange(0,1.01,0.01))

for column in ewr_fdcs:   
    (month,day)=column.split('-')
    ewr_fdcs[column]=np.percentile(daily[(daily['Month']==int(month)) & (daily['Day']==int(day))]['Flow'] ,(1-ewr_fdcs.index)*100,interpolation='linear')
    
ewr_fdcs

,9-28,10-8,10-18,10-28,11-8,11-18,11-28,12-8,12-18,12-28,...,6-18,6-28,7-8,7-18,7-28,8-8,8-18,8-28,9-8,9-18
0.00,444.787617,422.757761,444.787617,459.919828,490.160582,531.264912,629.767856,765.558486,815.702397,1127.620860,...,2618.401573,1988.009245,1551.841283,1190.622887,977.867489,754.867904,636.557731,568.266722,522.289492,476.721797
0.01,432.458894,422.455431,410.461602,425.428396,482.407401,517.816699,602.867691,701.544428,778.863303,1065.367011,...,2317.929624,1851.806471,1433.989612,1126.035847,920.674026,746.321971,616.055938,550.559704,492.730895,461.263986
0.02,391.213012,383.801250,381.408864,389.779781,431.917792,472.490269,577.860569,669.932970,764.799397,1006.595396,...,2061.142585,1609.792001,1230.495564,1000.675824,847.573901,703.914125,605.687774,531.572649,468.252396,428.427651
0.03,378.151708,359.286079,361.645286,363.353437,402.215320,444.639440,550.922328,638.216854,740.902809,914.416064,...,2011.595857,1559.106508,1182.458510,977.867489,769.657735,623.436412,560.970062,489.088410,444.639440,414.002455
0.04,376.443556,353.867929,339.428220,354.861279,387.487624,431.169947,501.587864,600.589954,707.013992,884.708251,...,1996.359539,1549.217407,1160.431821,933.054975,744.329702,593.867574,547.365094,476.721797,438.500503,405.046664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.96,179.747804,165.630893,148.811132,154.702178,154.691402,173.010587,200.601842,241.345489,276.423297,339.975662,...,416.142297,367.981714,341.414602,321.900903,291.013915,268.520799,234.817519,220.790945,202.743568,192.440410
0.97,169.881107,148.297489,138.940445,146.795479,148.665417,170.439890,194.443204,232.452444,265.750305,320.952985,...,387.785736,345.850477,327.753898,307.087715,284.766704,258.896041,230.438584,212.818697,199.804625,184.989573
0.98,152.677266,125.878777,116.912047,124.123539,142.747171,161.423169,190.505029,222.170543,260.335273,275.168456,...,339.881910,307.492983,287.681326,263.249807,259.712637,236.307631,219.825566,202.894567,190.831594,177.422958
0.99,138.797377,122.972481,112.155549,115.931495,138.622713,144.652165,186.563703,195.233840,235.930137,264.667848,...,325.272624,289.747717,269.319967,252.893518,231.400287,218.047310,197.461671,187.330502,165.998093,153.817529


In [347]:
daily['dDate']=daily.index
daily['MeasureDay']=daily.apply(lambda x: isMeasureDay(x['dDate']),axis=1)
ewr_measure_days=pd.DataFrame(daily.loc[daily['MeasureDay']!=False]['MeasureDay'])
ewr_measure_days['Flow']=daily['Flow']
ewr_measure_days['Exceedance']=ewr_measure_days.apply(lambda x: lookupExceedance(x['MeasureDay'],x['Flow']),axis=1)
ewr_measure_days

,MeasureDay,Flow,Exceedance
Date,,,
1924-10-08,10-8,111.133124,1.00
1924-10-18,10-18,111.133124,1.00
1924-10-28,10-28,125.539492,0.98
1924-11-08,11-8,139.095903,0.99
1924-11-18,11-18,144.946329,0.99
...,...,...,...
2017-08-18,8-18,309.012275,0.78
2017-08-28,8-28,274.574324,0.79
2017-09-08,9-8,240.615078,0.81


In [348]:
daily['dDate']=daily.index
daily['EWRMeasureDate']=daily.apply(lambda x: measureDate(x['dDate']),axis=1)

In [349]:
daily['EWRRefExceedance']=daily.join(ewr_measure_days,on='EWRMeasureDate',lsuffix='_l')['Exceedance']

In [350]:
daily=daily.drop(['MeasureDay','EWRMeasureDate','dDate'],axis=1)
daily=daily.fillna(0.5)
daily

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,Volume,EWRRefExceedance
Date,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124,0.999,1924,10,1,1924.10,1924,1,1,0.009602,0.50
1924-10-02,1924-10-13,100.0,1.111331,111.133124,0.999,1924,10,2,1924.10,1924,1,2,0.009602,0.50
1924-10-03,1924-10-14,100.0,1.111331,111.133124,0.999,1924,10,3,1924.10,1924,1,3,0.009602,0.50
1924-10-04,1924-10-15,100.0,1.111331,111.133124,0.999,1924,10,4,1924.10,1924,1,4,0.009602,0.50
1924-10-05,1924-10-16,100.0,1.111331,111.133124,0.999,1924,10,5,1924.10,1924,1,5,0.009602,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-26,2017-10-07,204.0,0.995184,203.017496,0.950,2017,9,26,2017.09,2016,12,361,0.017541,0.82
2017-09-27,2017-10-08,204.0,0.995184,203.017496,0.950,2017,9,27,2017.09,2016,12,362,0.017541,0.82
2017-09-28,2017-10-09,204.0,0.995184,203.017496,0.950,2017,9,28,2017.09,2016,12,363,0.017541,0.82


In [351]:
daily.to_csv('prepared_daily_ewrx.csv')